In [1]:
import Transform.Schedule

from Configuration import Editor

with Editor('Config') as Config:

    # settings for dataset
    Config.Dataset.ImagesRootPath = r'D:\Dataset_Collection\Cardiac_Catheterization\train\images'
    Config.Dataset.MasksRootPath = r'D:\Dataset_Collection\Cardiac_Catheterization\train\masks'
    # control input and output image format
    Config.Dataset.IO.InputRGBImage = False
    Config.Dataset.IO.NumWorkers = 0
    Config.Dataset.IO.PinMemory = False
    Config.Dataset.IO.PrefetchFactor = 2
    Config.Dataset.IO.OutputDtype = 'float'
    # uniformed preprocess
    Config.Dataset.Preprocess.Version = 'v1'
    #train dataset
    Config.Dataset.Train.BatchSize = 1
    Config.Dataset.Train.Transform.Combination.Version = 'v1'
    Config.Dataset.Train.Transform.Schedule = 0.8
    Config.Dataset.Train.Transform.Combination.Components = 'default'
    Config.Dataset.Train.Transform.Combination.Params = 'default'
    Config.Dataset.Train.Transform.Combination.Schedules = 'default'
    #validation dataset
    Config.Dataset.Validation.BatchSize = 8
    Config.Dataset.Validation.Ratio = 0.05
    Config.Dataset.Validation.Transform.Combination.Version = 'v1'
    Config.Dataset.Validation.Transform.Schedule = 0.8
    Config.Dataset.Validation.Transform.Combination.Components = 'default'
    Config.Dataset.Validation.Transform.Combination.Params = 'default'
    Config.Dataset.Validation.Transform.Combination.Schedules = 'default'
    
    #choose training structure, including the model, loss, metrics, optimizer, schedular
    Config.Training.Structure.Type = 'SimpleSeg'
    # model
    Config.Training.Structure.Model.Backbone.Name = 'smp_Unet'
    Config.Training.Structure.Model.Backbone.Param = dict(encoder_name='mobilenet_v2', encoder_weights='imagenet', in_channels=1, output_dim=32)
    Config.Training.Structure.Model.Head.Name = 'V1'
    Config.Training.Structure.Model.Head.Param = dict(in_channels=32)
    # loss
    Config.Training.Structure.Loss.Name = 'DiceLoss'
    Config.Training.Structure.Loss.Param = dict()
    # optimizer
    Config.Training.Structure.Optimizer.Name = 'Adam'
    Config.Training.Structure.Optimizer.Param = dict()
    # scheduler
    Config.Training.Structure.Scheduler.Name = 'LinearLR'
    Config.Training.Structure.Scheduler.Param = dict(warmup_epochs=2)
    # metrics
    Config.Training.Structure.Metrics.Name = ['DiceLoss']
    Config.Training.Structure.Metrics.Param = [dict()]

    # setup training
    Config.Training.Checkpoint.Path = None
    Config.Training.Checkpoint.FileName = ''
    Config.Training.Checkpoint.Resume.Process = False
    Config.Training.Checkpoint.Resume.Optimizer = False
    Config.Training.Checkpoint.Resume.Scheduler = False
    
    # whether to freeze backbone
    Config.Training.Settings.Model.FreezeBackbone = True
    
    # setup training process
    Config.Training.Settings.Epochs = 10
    Config.Training.Settings.GradientAccumulation = 4
    Config.Training.Settings.AmpScaleTrain = False
    # set random property
    Config.Training.Settings.Random.cuDNN.Deterministic = True
    Config.Training.Settings.Random.cuDNN.Benchmark = True
    Config.Training.Settings.Random.Seed.Dataset = 4
    Config.Training.Settings.Random.Seed.Model = 100
    
    Config.Logging.StepsPerLog = 8
    Config.Logging.Image.Columns = 3
    Config.Logging.Image.Rows = 10
    Config.Logging.Image.Figsize = (300,300)
    Config.Logging.Image.Fontsize = 200
    Config.Logging.Image.DPI = 10
    Config.Logging.Image.MaskAlpha = 0.3
    
    Config.Logging.RootPath = 'logging'
    Config.Logging.Model.Reference = 'SimpleSeg_FCN_ResNet'
    Config.Logging.Model.Derivative = 'Test'
    Config.Logging.Model.Branch = 'Test'
    Config.Logging.Comment = 'Test'
    Config.Logging.Purpose = 'Test'
    Config.Logging.Note = 'None'

In [2]:
import os
import io
import pathlib
import random
import math
import numpy as np

import torch
from torch.cuda.amp import autocast, GradScaler

import Configuration
from Dataset import TrainDataLoader
import Transform.Preprocess
import Transform.Combinations
import Transform.Schedule
import Structure
import utils.train
import utils.schedulers


Config = Configuration.Config

# setup environments
# create gpu device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# seed everything
# seed for dataset
dataset_seed = Config.Training.Settings.Random.Seed.Dataset
os.environ['PYTHONHASHSEED'] = str(dataset_seed)
random.seed(dataset_seed)
np.random.seed(dataset_seed)
# seed for model creation
model_seed = Config.Training.Settings.Random.Seed.Model
torch.manual_seed(model_seed)
torch.cuda.manual_seed(model_seed)
# cudnn reproducibility
torch.backends.cudnn.deterministic = Config.Training.Settings.Random.cuDNN.Deterministic
torch.backends.cudnn.benchmark = Config.Training.Settings.Random.cuDNN.Benchmark

# create recorder
recorder = utils.train.Recorder(
    root = Config.Logging.RootPath,
    reference = Config.Logging.Model.Reference,
    derivative = Config.Logging.Model.Derivative,
    branch = Config.Logging.Model.Branch,
    comment = Config.Logging.Comment,
    purpose = Config.Logging.Purpose,
    dataset_seed = dataset_seed,
    model_seed = model_seed)

# create dataset: train_dataset, validation_dataset, validation_dataset_wo_arg
# train_transform for dataloader
train_transform_creater = getattr(Transform.Combinations, Config.Dataset.Train.Transform.Combination.Version)
train_transform_creater = train_transform_creater(
    Config.Dataset.Train.Transform.Schedule,
    Config.Dataset.Train.Transform.Combination.Components,
    Config.Dataset.Train.Transform.Combination.Params,
    Config.Dataset.Train.Transform.Combination.Schedules)
# validation_transform for dataloader
validation_transform_creater = getattr(Transform.Combinations, Config.Dataset.Validation.Transform.Combination.Version)
validation_transform_creater = validation_transform_creater(
    Config.Dataset.Validation.Transform.Schedule,
    Config.Dataset.Validation.Transform.Combination.Components,
    Config.Dataset.Validation.Transform.Combination.Params,
    Config.Dataset.Validation.Transform.Combination.Schedules)
# standard preprocess operation for dataloader
preprocess = getattr(Transform.Preprocess,Config.Dataset.Preprocess.Version)
# create dataloader
dataloader = TrainDataLoader(
    images_root = Config.Dataset.ImagesRootPath,
    masks_root = Config.Dataset.MasksRootPath,
    train_transform = train_transform_creater,
    train_batch_size = Config.Dataset.Train.BatchSize,
    validation_ratio = Config.Dataset.Validation.Ratio,
    validation_transform = validation_transform_creater,
    validation_batch_size = Config.Dataset.Validation.BatchSize,
    image_rgb = Config.Dataset.IO.InputRGBImage,
    preprocess = preprocess,
    num_workers = Config.Dataset.IO.NumWorkers,
    pin_memory = Config.Dataset.IO.PinMemory,
    prefetch_factor = Config.Dataset.IO.PrefetchFactor,
    dtype=Config.Dataset.IO.OutputDtype)
# get datasets from dataloader
train_dataset, validation_dataset, validation_dataset_wo_arg = dataloader.get_dataset()

# calcualte how many steps in an epoch and scheduler need this
train_data_count = len(train_dataset)
gradient_accumulation = Config.Training.Settings.GradientAccumulation
train_batch_size = Config.Dataset.Train.BatchSize
step_size =  gradient_accumulation * train_batch_size
steps_per_epoch = int(math.ceil(train_data_count/step_size))

# Select the training structure and corresponding to model, loss_fn, metrics
structure = getattr(Structure,Config.Training.Structure.Type)
# start building the model
# build the model backbone
model_backbone_class = getattr(structure.Model.Backbones,Config.Training.Structure.Model.Backbone.Name)
model_backbone_param = Config.Training.Structure.Model.Backbone.Param
model_backbone = model_backbone_class(**model_backbone_param)
# build the model head
model_head_class = getattr(structure.Model.Heads,Config.Training.Structure.Model.Head.Name)
model_head_param = Config.Training.Structure.Model.Head.Param
model_head = model_head_class(**model_head_param)
# create the model
model = utils.train.ModelBuilder(model_backbone,model_head)
model = model.to(device)
# setup loss_fn
loss_fn_class = getattr(structure.Losses,Config.Training.Structure.Loss.Name)
loss_fn = loss_fn_class(**Config.Training.Structure.Loss.Param)
# setup optimizer
optimizer_class = getattr(torch.optim,Config.Training.Structure.Optimizer.Name)
optimizer = optimizer_class(model.parameters(),**Config.Training.Structure.Optimizer.Param)
# scheduler
scheduler_class = getattr(utils.schedulers,Config.Training.Structure.Scheduler.Name)
scheduler = scheduler_class(steps_per_epoch=steps_per_epoch,**Config.Training.Structure.Scheduler.Param)
scheduler = scheduler(optimizer)
# setup metrics and setup recoder
metrics_class = [getattr(structure.Metrics,m) for m in Config.Training.Structure.Metrics.Name]
metrics_params = Config.Training.Structure.Metrics.Param
recorder.create_metrics_and_writers(metrics_class,metrics_params)

recorder.log_config(Config)

# initialize process state or resume checkpoint
start_epoch = 0
global_step = 0
images_count = 0
checkpoint_path = Config.Training.Checkpoint.Path
checkpoint_file = Config.Training.Checkpoint.FileName
steps_per_log = Config.Logging.StepsPerLog
if (checkpoint_path != None) or (checkpoint_path == ''):
    checkpoint_file_path = pathlib.Path(checkpoint_path,checkpoint_file).as_posix()
    checkpoint = torch.load(checkpoint_file_path)
    
    model_state_dict = checkpoint['model']
    model.load_state_dict(model_state_dict)
    
    images_count = checkpoint['images_count']
    global_step = checkpoint['global_step']
    
    if Config.Training.Checkpoint.Resume.Process:
        start_epoch = checkpoint['epoch']
    
    if Config.Training.Checkpoint.Resume.Optimizer:
        optimizer.load_state_dict(checkpoint['optimizer'])
        
    if Config.Training.Checkpoint.Resume.Scheduler:
        scheduler.load_state_dict(checkpoint['scheduler'])
    
# if the model's backbone is not going to be trained, but still keep the input and the output layers trainable. 
if Config.Training.Settings.Model.FreezeBackbone:
    model.freeze_backbone()

# setup for logging validation image segmentation examples
ncols = Config.Logging.Image.Columns * 3
nrows = Config.Logging.Image.Rows
figsize = Config.Logging.Image.Figsize
fontsize = Config.Logging.Image.Fontsize
dpi = Config.Logging.Image.DPI
mask_alpha = Config.Logging.Image.MaskAlpha

amp_scale_train = Config.Training.Settings.AmpScaleTrain
if amp_scale_train:
    scaler = GradScaler()

# start training process
end_epoch = start_epoch + Config.Training.Settings.Epochs
for epoch in range(start_epoch, end_epoch):
    print(f'==========epoch {epoch}=============')
    print('=====Train=====')
    acc_count = 0
    acc_loss = 0
    # set model to train mode
    model.train()
    for batch_train_data in train_dataset(epoch):
        images,masks = batch_train_data
        images = images.to(device)
        masks = masks.to(device)
        if amp_scale_train:
            with autocast():
                predicts = model(images)
                loss = loss_fn(predicts,masks)
        else:
            predicts = model(images)
            loss = loss_fn(predicts,masks)
        
        recorder.update_metrics_state(purpose='train', predict=predicts, label=masks)
        
        # update process state
        images_count += images.shape[0]
        acc_loss += loss
        acc_count += 1
        
        # if it's time to do back propagation
        if acc_count == gradient_accumulation:
            acc_loss = acc_loss / gradient_accumulation
            
            if amp_scale_train:
                scaler.scale(acc_loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                acc_loss.backward()
                optimizer.step()
            
            optimizer.zero_gradient()
            scheduler.step()
            acc_loss = 0
            acc_count = 0
            global_step += 1
        
        if (global_step % steps_per_log == 0) and global_step != 0:
            recorder.log_metrics_state(purpose='train',images_count=images_count)
            recorder.reset_metrics_state(purpose='train')
            recorder.log_lr(lr=optimizer.param_groups[0]['lr'],images_count=images_count)
    
    if global_step % steps_per_log != 0:
        recorder.log_metrics_state(purpose='train',images_count=images_count)
        recorder.reset_metrics_state(purpose='train')
        recorder.log_lr(lr=optimizer.param_groups[0]['lr'],images_count=images_count)
    
    if acc_count != 0:
        acc_loss = acc_loss / acc_count
        if amp_scale_train:
            scaler.scale(acc_loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            acc_loss.backward()
            optimizer.step()
        scheduler.step()
        optimizer.zero_gradient()
        acc_loss = 0
        acc_count = 0
        global_step += 1
    
    model.eval()
    for purpose, val_dataset in zip(('validation','validation_wo_arg'),(validation_dataset, validation_dataset_wo_arg)):
        print(f'====={purpose}=====')
        
        acc_output_images = 1
        fig = plt.figure(figsize=figsize,dpi=dpi)
        plt.axis(False)
        
        for batch_val_data in val_dataset(epoch):
            images, masks = batch_val_data
            images = images.to(device)
            masks = masks.to(device)
            predicts = model(images)
            
            if acc_output_images <= nrows*ncols:
                for image,mask,predict in zip(images.cpu().numpy(),masks.cpu().numpy(),predicts.cpu().numpy()):
                    if acc_output_images < nrows*ncols:
                        fig.add_subplot(nrows,ncols,acc_output_images)
                        plt.imshow(image,cmap='gray')
                        plt.title('Input Image',fontsize=fontsize)
                        plt.axis(False)
                        acc_output_images += 1
                        
                        fig.add_subplot(nrows,ncols,acc_output_images)
                        plt.imshow(image,cmap='gray')
                        plt.imshow(mask,cmap='gray',alpha=mask_alpha)
                        plt.title('Mask',fontsize=fontsize)
                        plt.axis(False)
                        acc_output_images += 1
                        
                        fig.add_subplot(nrows,ncols,acc_output_images)
                        plt.imshow(image,cmap='gray')
                        plt.imshow(predict,cmap='gray',alpha=mask_alpha)
                        plt.title('Predict',fontsize=fontsize)
                        plt.axis(False)
                        acc_output_images += 1
            
            recorder.update_metrics_state(purpose=purpose, predict=predicts, label=masks)
        
        scheduler.epoch(recorder)
        
        recorder.log_metrics_state(purpose=purpose,images_count=images_count)
        checkpoint = dict(
            model = model.state_dict(),
            optimizer = optimizer.state_dict(),
            scheduler = scheduler.state_dict(),
            epoch = epoch,
            global_step = global_step,
            images_count = images_count)
        recorder.save_checkpoint(purpose=purpose,checkpoint=checkpoint)
        recorder.reset_metrics_state(purpose='train')
        
        #record image
        plt.tight_layout()
        buf = io.BytesIO()
        plt.savefig(buf, format='raw', dpi=dpi)
        plt.close()
        buf.seek(0)
        img_arr = np.reshape(np.frombuffer(buf.getvalue(), dtype=np.uint8),
                            newshape=(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), -1))
        buf.close()
        recorder.log_image(purpose=purpose,image=img_arr,epoch=epoch)

==========epoch 0=============
=====Train=====


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 2.00 GiB total capacity; 1.10 GiB already allocated; 0 bytes free; 1.11 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
model

NameError: name 'model' is not defined